### Configure chain internals at runtime

In [2]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

In [3]:
model.invoke("pick a random number")

AIMessage(content='7')

In [4]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("pick a random number")

AIMessage(content='37')

In [6]:
prompt = PromptTemplate.from_template("Pick a random number above {x}")
chain = prompt | model

In [7]:
chain.invoke({"x": 0})

AIMessage(content='57')

In [8]:
chain.with_config(configurable={"llm_temperature": 0.9}).invoke({"x": 0})

AIMessage(content='67')

### With HubRunnables

In [9]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub commit to pull from",
    )
)

prompt.invoke({"question": "foo", "context": "bar"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: foo \nContext: bar \nAnswer:")])

In [10]:
prompt.with_config(configurable={"hub_commit": "rlm/rag-prompt-llama"}).invoke(
    {"question": "foo", "context": "bar"}
)

ChatPromptValue(messages=[HumanMessage(content="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: foo \nContext: bar \nAnswer: [/INST]")])

### Configurable Alternatives

#### With LLMs

In [14]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(temperature=0).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatOpenAI(),
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(),
    # You can add more configuration options here
)
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm

In [15]:
# By default it will call Anthropic
chain.invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

### With Prompts

In [17]:
llm = ChatOpenAI(temperature=0)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

In [18]:
# By default it will write a joke
chain.invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

In [19]:
# We can configure it write a poem
chain.with_config(configurable={"prompt": "poem"}).invoke({"topic": "bears"})

AIMessage(content="In the wild, where nature thrives,\nA creature roams with gentle strides.\nWith fur so thick, and claws so strong,\nThe mighty bear, where it belongs.\n\nIn forests deep, where shadows play,\nThe bear emerges, night or day.\nIts presence felt, a force untamed,\nA symbol of strength, never maimed.\n\nWith eyes so wise, and heart so pure,\nThe bear endures, forever sure.\nThrough winters harsh, it finds its way,\nIn solitude, it learns to sway.\n\nA guardian of the wild and free,\nThe bear roams with dignity.\nFrom mountains high to rivers wide,\nIt teaches us to stand with pride.\n\nYet, in its might, a tender side,\nA mother's love, it can't hide.\nCubs in tow, they learn to roam,\nGuided by a love called home.\n\nSo let us honor this noble beast,\nWith reverence, let our hearts feast.\nFor in the bear, we find a grace,\nA reminder of nature's embrace.")

### With Prompts and LLMs

In [20]:
llm = ChatOpenAI(temperature=0).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatOpenAI(),
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(),
    # You can add more configuration options here
)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

In [21]:
# We can configure it write a poem with OpenAI
chain.with_config(configurable={"prompt": "poem", "llm": "openai"}).invoke(
    {"topic": "bears"}
)

AIMessage(content="In the forest's deep embrace,\nA creature roams with gentle grace.\nWith fur of gold and eyes so wise,\nThe bear, majestic, we idolize.\n\nThrough rugged lands, it strides so strong,\nIn solitude, where it belongs.\nA symbol of both might and peace,\nIts presence brings a sweet release.\n\nWith paws that cast a mighty shadow,\nAnd strength that few could ever know,\nThe bear, a guardian of the wild,\nProtecting its young, undefiled.\n\nFrom rivers teeming with salmon's might,\nTo mountains tall, kissed by sunlight,\nThe bear roams freely, without a care,\nA symbol of nature's love affair.\n\nOh, bear, your spirit fierce and true,\nWe honor all that dwells in you.\nA reminder of the wild and free,\nA symbol of what we strive to be.")

In [22]:
# We can always just configure only one if we want
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

#### Saving configurations

In [ ]:
openai_poem = chain.with_config(configurable={"llm": "openai"})
openai_poem

In [24]:
openai_poem.invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")